In [1]:
import torch
import copy
import torch.nn.functional as F
from models.resnet_cifar10 import ResNet18_CIFAR10
from models.module import QLinear
def cmp(a,b):
    return (a-b).abs().max()
model = ResNet18_CIFAR10()
model.eval()
model.fuse_bn()
model.quantize(w_num_bits=4,a_num_bits=4)
model.load_state_dict(torch.load('./qat_logs/resnet_cifar10_w4a4/checkpoint_max_w4a4.pth')['net'])
q_model = copy.deepcopy(model)
q_model.freeze()


q_model.linear.linear_module.bias

Parameter containing:
tensor([ -5.,   4.,  -5.,  13., -10.,   9.,   3., -14., -21.,   2.],
       requires_grad=True)

In [2]:
dummy_input = model.layer4[-1].add.qo.fake_quantize(torch.rand([1000,8192]))
q_input = q_model.layer4[-1].qo.quantize_tensor(dummy_input)

out = model.linear(dummy_input, qi=q_model.layer4[-1].qo)
q_out = q_model.linear.qo.dequantize_tensor(q_model.linear.quantize_inference(q_input))
cmp(out, q_out)

tensor(0.9553, grad_fn=<MaxBackward1>)

In [3]:
x = q_model.layer4[-1].add.qo.fake_quantize(dummy_input)
(x==dummy_input).sum()


tensor(8192000)

In [4]:
q_model.linear

tensor([[-7.6420, -6.6868, -6.6868, -7.6420, -7.6420, -7.6420, -7.6420, -7.6420,
         -7.6420, -7.6420]], grad_fn=<MulBackward0>)

In [5]:
q_out

tensor([[-7.6420, -6.6868, -5.7315, -7.6420, -7.6420, -6.6868, -7.6420, -7.6420,
         -7.6420, -7.6420]], grad_fn=<MulBackward0>)

In [14]:
q_model.linear.qo

QParam()

In [5]:
q_model.layer1[0].conv1.n

tensor(10.)